# Higher-order analysis of path data in `pathpy`


**June 12 2020**


In [2]:
import pathpy as pp
import pandas as pd
import numpy as np
import scipy.stats
from matplotlib import pyplot as plt

# 1. Temporal Network Represenation

The key idea is that the ordering and timing in which time-stamped edges occur in a `TemporalNetwork` gives rise to so-called **causal or time-respecting paths**. In a nutshell, for two time-stamped edges $(a, b, t)$ and $(b, c, t')$ to contribute to a causal path $a \rightarrow b \rightarrow c$ it must hold that $t < t'$. If we swap timestamps such that the edge $(b, c)$ occurs **before** (a,b), no causal path $a \rightarrow b \rightarrow c$ exists.

So we observe that the chronological order of time-stamped edges crucially influences causal paths, i.e. which nodes can possibly influence each other in time-stamped edge sequences. Moreover, we often want to limit the **maximum time difference between consecutive edges** that contribute to a causal path. For data on dynamic social interactions that spans several years, it does not make sense to consider all chronologically ordered edges as possible causal paths for the propagation of information. After all, humans have limited memory and we should thus consider interactions that occur far apart in time as independent.

We can formally add this condition by setting a maximum time difference $\delta$ for the path calculation. That is, we only consider two edges $(a, b, t)$ and $(b, c, t')$ to contribute to a causal path $a \rightarrow b \rightarrow c$ if $ 0 < t' - t \leq \delta$.

With this definition at hand, and by setting our maximum time difference $\delta$, we can now **calculate causal path statistics in time-stamped network data**. `pathpy` provides powerful algorithms to calculate (or estimate) causal path statistics based on a `TemporalNetwork` instance. Let us try this in our toy example.

In [ ]:
patients = pd.read_csv('./Patient Data/CPE_temporal_edges.csv')

In [ ]:
print(patients)

In [ ]:
t = pp.TemporalNetwork()

for index, row in patients.iterrows():
    t.add_edge(row[0], row[1], row[2])

In [ ]:
print(t)

In [ ]:
style = {    
  'ts_per_frame': 7, 
  'ms_per_frame': 150,
  'look_ahead': 200, 
  'look_behind': 200, 
  'node_size': 8, 
  'inactive_edge_width': 2,
  'active_edge_width': 4, 
  'label_color' : 'black',
  'label_size' : '5px',
  'label_offset': [0,0], 
    'width': 800, 'height': 500
  }

pp.visualisation.plot(t, **style)

In [ ]:
pp.visualisation.export_html(t, 'CPE_temporal_network.html', **style)

In [ ]:
causal_paths = pp.path_extraction.temporal_paths.paths_from_temporal_network_dag(t, delta=7)
print(causal_paths)

In [ ]:
for l in causal_paths.paths:
    for p in causal_paths.paths[l]:
        if causal_paths.paths[l][p][1]>0:
            print('{0} -> {1}'.format(p, causal_paths.paths[l][p][1]))

## 2. Higher order Models from actual pathway data

The data analysis and modelling framework outlined in these works builds on a generalisation of standard, first-order networks to $k$-dimensional De Bruijn graph models for paths in complex networks.

Note: De Bruijn graph of $m$ symbols is a directed graph representing overlaps between sequences of symbols. 

The class `HigherOrderNetwork` allows us to generate such higher-order network models of paths. In the documentation, we find that the constructor takes a parameter `paths`, i.e. the statistics of the observed paths that we want to model. With the parameter `k` we specify the order $k$ of the higher-order model that we want to fit. To understand this better, let us do this for our toy example.

In [ ]:
paths = pd.read_csv('./Patient Data/CPE_pathways.csv') 

patientPaths = pp.Paths()

for index, row in paths.iterrows():
    patientPaths.add_path(row[0],  separator='-', frequency=1)

#patientPaths *= 8

#patientPaths = causal_paths

This output confirms that a `HigherOrderModel` with $k=1$ is identical to our `Network` model. WIth one exception: edge weights are vector-valued. Just like in `Paths`, the first entry captures the sub-path frequency while the second entry counts the occurrence of an edge as a longest path. 

We can see this network as a **first-order model** for paths where **edges are paths of length one**. That is, in a model with order $k=1$ edge weights capture the statistics of paths of length $k=1$.

In [ ]:
hon_1 = pp.HigherOrderNetwork(patientPaths, k=1)
print(hon_1)

In [ ]:
style = { 'label_offset': [0,-1], 'label_color' : 'black', 'width': 800, 'height': 500,'label_size' : '5px'}
pp.visualisation.plot(hon_1, **style)

We can generalise this idea to **k-th-order models** for paths, where **nodes are paths of length $k-1$** while **edge weights capture the statistics of paths of length $k$**. We can generate such a $k$-th order model by performing a line graph transformation on a model with order $k-1$. That is, edges in the model of order $k-1$ become nodes in the model with order $k$. We then draw edges between higher-order nodes whenever there is a possible path of length $k$ in the underlying network. The result is a $k$-dimensional De Bruijn graph model for paths. Let us try this in our example.

In [ ]:
hon_2 = pp.HigherOrderNetwork(patientPaths, k=2)
#pp.visualisation.plot(hon_2, **style)

Each of the **edges** in the first-order model is now represented by a **node** in the second-order model.

This is important because it captures to what extent the paths that we observe in our data deviate from what we would expect based on the (first-order) network topology of the system. Considering such a first-order model, all paths of length two are possible. If edges were statistically independent we would expect all paths f length 2 to occur with the same frequency.

Another way to express this independence assumption is to consider Markov chain models for the sequences of nodes traversed by a path. In this view, independently occurring edges translate to a memoryless first-order Markov process for the node sequence. 

`pathpy` can actually generate this **null-model** for paths within the space of possible second-order models. This allows us to  compare how the observed path statistics deviate from a (Markovian) expectation.

In [ ]:
hon_2_null = pp.HigherOrderNetwork(patientPaths, k=2, null_model=True)
#pp.visualisation.plot(hon_2_null, **style)

#for e in hon_2_null.edges:
#    print(e, hon_2_null.edges[e])

We can easily find out which of the paths of length two occur more or less often than expected under the null model. We can just subtract the adjacency matrices of the two instances `hon_2` and `hon_2_null`.

In [ ]:
A_2 = hon_2.adjacency_matrix()
idx_2 = hon_2.node_to_name_map()

A_2n = hon_2_null.adjacency_matrix()
idx_2n = hon_2_null.node_to_name_map()

#for (v,w) in hon_2_null.edges:
#    print(e, A_2[idx_2[v],idx_2[w]] - A_2n[idx_2n[v],idx_2n[w]])

This analysis confirms that the paths a number of paths occur more times than we would expect at random, while the other paths do occur less times than expected. Importantly, this deviation from our expectation changes the causal topology of the system, i.e. who can influence whom. In a network model we implicitly assume that paths are transitive, i.e. since $a$ is connected to $c$ and $c$ is connected to $d$ we assume that there is a path by which $a$ can influence $d$ via node $c$. The second-order model of our data reveals that this transitivity assumption is misleading, highlighting higher-order dependencies in our data that result in the fact that neither some $a$ can influence some $d$ nor some $b$ can always influence some $e$.


### Ranking nodes in higher-order networks

In summary, higher-order models capture deviations from the transitivity assumption that we often implicitly make when we apply standard graph mining or network analysis techniques. An important class of methods that rely on this assumption are centrality measures, which are often used to rank nodes in networked systems.

Let us consider the betweenness value of a node $v$, which (in its unnormalized variant) captures for how many pairs of nodes $x,y$ the shortest path from $x$ to $y$ passes through $v$. This measure is important, because is teaches us something about the role of nodes in information flow. It further captures to what extent removing a node will affect the shortest paths between other nodes. Let us try this in our example:

In [ ]:
print(pp.algorithms.centralities.betweenness(hon_1)['SM_ZCO'])

However, this notion of *importance* of node $c$ is based on the assumption that paths are transitive and Markovian, which is not the case in our observed paths. `pathpy` actually allows us to calculate the betweenness of node $c$ in the observed paths

In [ ]:
print(pp.algorithms.centralities.betweenness(patientPaths)['SM_ZCO'])

Not surprisingly, the betweenness of the node in the actual paths is different. This change in centrality can be captured in the higher-order model and `pathpy` allows us to directly generalize centrality measures to higher orders, simply by calling the functions on the class `HigherOrderNetwork`.

In [ ]:
print(pp.algorithms.centralities.betweenness(hon_2)['SM_ZCO'])

In [ ]:
print(pp.algorithms.centralities.betweenness(hon_2_null)['SM_ZCO'])

In [ ]:
pageRank_hon1 = pp.algorithms.centralities.pagerank(hon_1)
pageRank_hon2 = pp.algorithms.centralities.pagerank(hon_2)
pageRank_hon2_null = pp.algorithms.centralities.pagerank(hon_2_null)

## Predicting node rankings with higher-order models

Let us now explore how we can apply higher-order models in practical prediction tasks. Assume we are given the task to rank nodes in a network according to how often they are traversed by paths. This has practical applications, e.g. predicting the most frequently clicked Web pages based on the topology of the link graph, or predicting the most congested airports based on the topology of flight connections. Let us study the latter example based on a data set on the paths of airline passengers in the United Stated.

For your convenience, we have partitioned the data set into a training and a validation set of equal size. Our goal will be to use the training set to learn (higher-order) network models. We then calculate a higher-order generalisation of `pagerank` described [in this paper](http://dl.acm.org/citation.cfm?id=3098145) in these models to predict the ranking of airports according to the number of passengers in the validation set.

<span style="color:red">**TODO:** </span>

In [ ]:
# Split Paths into test and train
#paths = pd.read_csv('./Patient Data/CPE_pathways_more2moves.csv') 
paths = pd.read_csv('./Patient Data/CPE_pathways.csv') 
msk = np.random.rand(len(paths)) < 0.6
train, test = paths[msk], paths[~msk]
trainPaths = pp.Paths()
testPaths = pp.Paths()

for index, row in train.iterrows():
    trainPaths.add_path(row[0],  separator='-', frequency=1)

for index, row in test.iterrows():
    testPaths.add_path(row[0],  separator='-', frequency=1)
    
#gt = pp.algorithms.centralities.pagerank(pp.HigherOrderNetwork(testPaths, k=2))
gt = pp.algorithms.centralities.node_traversals(testPaths)

We want to evaluate the goodness of this prediction (in terms of the node ranking) using the Kendall-Tau rank correlation coefficient. Since the overlap between the nodes observed in the training data, and the nodes observed in the validation data may not be perfect, we first need a function that calculates the Kendall-Tau correlation for the intersection of nodes in the training and validation sets.

In [ ]:
def kendalltau(a, b):
    x = []
    y = []
    for v in set(a.keys()).intersection(set(b.keys())):
            x.append(a[v])
            y.append(b[v])
    return scipy.stats.kendalltau(x, y)


def validate(mog, gt):
    res = {}
    for k in range(1, mog.max_order+1):
        pr = pp.algorithms.centralities.pagerank(mog.layers[k])
        res[k] = kendalltau(gt, pr).correlation
    return res

def plot_results(res, optimal_order):
    if optimal_order>1:
        diff = res[optimal_order] - res[1]
        print('Relative increase over first-order = {0} %'.format(100*diff/res[1]))
    plt.plot(range(1, max(res.keys())+1), [res[k] for k in range(1, max(res.keys())+1)], '-o')
    #plt.plot([optimal_order], [res[optimal_order]], 'g', marker='x', markersize=20.0)
    plt.title('Kendall-Tau')
    plt.show()

In [ ]:
mog = pp.MultiOrderModel(trainPaths, 6)
order = mog.estimate_order()

res = validate(mog, gt)
plot_results(res, order)

In [ ]:
res = []

for x in range(6):
    msk = np.random.rand(len(paths)) < 0.6
    train, test = paths[msk], paths[~msk]
    trainPaths = pp.Paths()
    testPaths = pp.Paths()

    for index, row in train.iterrows():
        trainPaths.add_path(row[0],  separator='-', frequency=1)

    for index, row in test.iterrows():
        testPaths.add_path(row[0],  separator='-', frequency=1)
    
    gt = pp.algorithms.centralities.node_traversals(testPaths)
    
    l_mog = pp.MultiOrderModel(trainPaths, 6)
    l_order = mog.estimate_order()

    l_res = validate(mog, gt)
    
    res.append(l_res) 

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(1, max(res[0].keys())+1), [res[0][k] for k in range(1, max(res[0].keys())+1)], '-o')
plt.plot(range(1, max(res[1].keys())+1), [res[1][k] for k in range(1, max(res[1].keys())+1)], '-o')
plt.plot(range(1, max(res[2].keys())+1), [res[2][k] for k in range(1, max(res[2].keys())+1)], '-o')
plt.plot(range(1, max(res[3].keys())+1), [res[3][k] for k in range(1, max(res[3].keys())+1)], '-o')
plt.plot(range(1, max(res[4].keys())+1), [res[4][k] for k in range(1, max(res[4].keys())+1)], '-o')
plt.plot(range(1, max(res[5].keys())+1), [res[5][k] for k in range(1, max(res[5].keys())+1)], '-o')

plt.legend(['Boot0','Boot1', 'Boot2', 'Boot3', 'Boot4','Boot5'], loc='upper left')

plt.title('Kendall-Tau Rank Correlation')
plt.show()

In [ ]:
mog = pp.MultiOrderModel(trainPaths, 6)
order = mog.estimate_order()

# Clustering Multi-order Network

In [3]:
paths = pd.read_csv('./Patient Data/CPE_pathways.csv') 

patientPaths = pp.Paths()

for index, row in paths.iterrows():
    patientPaths.add_path(row[0],  separator='-', frequency=1)

# Multi-order Representation Learning

So far, we have studied higher-order network models for path data with a fixed, given order $k$. We have seen that such higher-order models can yield better predictions compared to standard network models. But we have also come across some open questions. For the US Flight data, we have seen that we could improve the prediction performance by moving from a first- to a second-order model. However, we did not see a further improvement for the third-order model. So in practice, how can we decide **at which order we should model a given system**? In fact, this points to a more general question. It is easy to imagine systems for which path statistics do not deviate significantly from the transitive, Markovian assumption made by a first-order network model. So we **need a methods to decide when higher-order models are actually needed**.

Moreover, a higher-order model with order $k$ can only capture higher-order dependencies at a single fixed correlation length $k$. But we may encounter data that exhibit multiple correlation lengths at once. This raises the question how we  can combine models with multiple higher orders into a multi-order model.

In this session, we take a statistical inference and machine learning perspective to answer these questions. Let us again start with a simple toy example:

 Let us take a statistical inference perspective on the problem. More specifically, we will consider our higher-order networks as probabilistic generative models for paths in a given network topology. For this, let us use the weighted first-order network model to construct a transition matrix of a Markov chain model for paths in a network. We simply use the relative frequencies of edges to proportionally scale the probabilities of edge transitions in the model.

This transition matrix can be viewed as a first-order Markov chain model for paths in the underlying network topology. This probabilistic view allows us to calculate a likelihood of the first-order model, given the paths that we have observed. With pathpy, we can directly calculate the likelihood of higher-order models.

In [ ]:
print(hon_2.likelihood(patientPaths, log=True))

In [ ]:
hon_2_null = pp.HigherOrderNetwork(patientPaths, k=2, null_model=True)
#pp.visualisation.plot(hon_2_null)
#print(hon_2.transition_matrix())
hon_2_null.likelihood(patientPaths, log=True)

### Model selection for higher-order network models

### Model selection for higher-order network models

This confirms our expectation that the second-order null model actually has the same likelihood as the first-order model. It also highlights an interesting way to test hypotheses about higher-order correlations in paths. We can use a likelihood ratio test to compare the likelihood of the null hypothesis (i.e. a second-order representation of the first-order model) with the likelihood of an alternative hypothesis (the *fitted* second-order model).

But what do we learn from th fact that the likelihood of a model increases as we increase the order of the model. By itself, not much. The reason for this is that higher-order models are typicall more complex than first-order models, i.e. while fitting their transition matrix, we actually fit more parameters to the data. We can thus expect that they better explain our observations. 

We should remind ourselves about Occam's razor, which states that we should favor models that make fewer assumptions. That is, in the comparison of the model likelihoods we should account for the additional complexity (or degrees of freedom) of a higher-order model over the null hypothesis.

A nice feature of our framework is that the null model and the alternative model are actually **nested**, i.e. the null model is one particular point in the parameter space of the (more general) higher-order model. Thanks to this property, we can apply [Wilk's theorem](https://en.wikipedia.org/wiki/Likelihood-ratio_test#Distribution:_Wilks’_theorem) to derive an analytical expression for the $p$-value of the null hypothesis that second-order correlations are absent (i.e. that a first-order model is sufficient to explain the observed paths), compared to the alternative hypothesis that a second-order model is needed. You can find the mathematical details of this hypothesis testing technique in the following KDD'17 paper:

I Scholtes: [When is a Network a Network? Multi-Order Graphical Model Selection in Pathways and Temporal Networks](http://dl.acm.org/citation.cfm?id=3098145), In KDD'17 - Proceedings of the 23rd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, Halifax, Nova Scotia, Canada, August 13-17, 2017

Let us now apply this likelihood ratio test the hypothesis that there are significant second-order dependencies in the paths of our toy example. The test consists of three basic steps: 

1. We calculate the difference $d$ between the degrees of freedom of a second- and a first-order model.
2. We calculate the test statistic $x = -2 \cdot(\log(\text{hon_1.likelihood}) - \log(\text{hon_2.likelihood}))$ for the likelihood ratio test.
3. We calculate a p-value as $1-cdf(x, d)$, where $cdf$ is the cumulative distribution function of a chi-square distribution.

In [ ]:
from scipy.stats import chi2

d = hon_2.degrees_of_freedom() - hon_1.degrees_of_freedom()
x = - 2 * (hon_1.likelihood(patientPaths, log=True) - hon_2.likelihood(patientPaths, log=True))
p = 1 - chi2.cdf(x, d)

print('The p-value of the null hypothesis (first-order model) is {0}'.format(p))

### Multi-order representation learning

To fix this issue, we need probabilistic generative models that can deal with large collections of (short) paths in a network. The key idea is to combine multiple higher-order network models into a single multi-layered, multi-order model. To calculate the likelihood of such a model based on a set of observed paths, we can use all layers, thus avoiding the problem that we discard prefixes of paths. For each path, we start at a layer of order zero, which considers the relative probabilities of nodes. We use this model layer to calculate the probability to observe the first node on a path. For the next transition to step two, we then use a first-order model. The next transition is calculated in the second-order model and so on, until we have reached the maximum order of our multi-order model. At this point, we can transitively calculate the likelihood based on the remaining transitions of the path.

The method is described, and illustrated with an example, in the following paper:

I Scholtes: [When is a Network a Network? Multi-Order Graphical Model Selection in Pathways and Temporal Networks](http://dl.acm.org/citation.cfm?id=3098145), In KDD'17 - Proceedings of the 23rd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, Halifax, Nova Scotia, Canada, August 13-17, 2017

`pathpy` can directly generate, visualise, and analyse multi-order network models. Let us try this in our example.

In [ ]:
mog = pp.MultiOrderModel(patientPaths, max_order=3)
print(mog)

#pp.visualisation.plot(mog.layers[0])
#pp.visualisation.plot(mog.layers[1])
#pp.visualisation.plot(mog.layers[2])

Let us now use the likelihood function of the class MultiOrderModel to repeat our likelihood ratio test. Rather than generating multiple MultiOrderModel instances for different hypotheses, we can directly calculate likelihoods based on different model layers within the same MultiOrderModel instance.

In [ ]:
mog = pp.MultiOrderModel(patientPaths, max_order=3)

d = mog.degrees_of_freedom(max_order=2) - mog.degrees_of_freedom(max_order=1)
x = - 2 * (mog.likelihood(patientPaths, log=True, max_order=1) - mog.likelihood(patientPaths, log=True, max_order=3))
p = 1 - chi2.cdf(x, d)

print('p value of null hypothesis that data has maximum order 1 = {0}'.format(p))

Rather than performing the likelihood test ourselves, we can actually simply call the method `MultiOrderModel.estimate_order`. it will return the maximum order among all of its layers for which the likelihood ratio test rejects the null hypothesis.

In [ ]:
mog.estimate_order()